# Data Preprocessing

The next step is to

In [1]:
#@title Imports for Project
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
df = pd.read_csv('../data/rideshare_kaggle.csv')
df.head()

,id,timestamp,hour,day,month,datetime,timezone,source,destination,cab_type,...,precipIntensityMax,uvIndexTime,temperatureMin,temperatureMinTime,temperatureMax,temperatureMaxTime,apparentTemperatureMin,apparentTemperatureMinTime,apparentTemperatureMax,apparentTemperatureMaxTime
0,424553bb-7174-41ea-aeb4-fe06d4f4b9d7,1.544953e+09,9,16,12,2018-12-16 09:30:07,America/New_York,Haymarket Square,North Station,Lyft,...,0.1276,1544979600,39.89,1545012000,43.68,1544968800,33.73,1545012000,38.07,1544958000
1,4bd23055-6827-41c6-b23b-3c491f24e74d,1.543284e+09,2,27,11,2018-11-27 02:00:23,America/New_York,Haymarket Square,North Station,Lyft,...,0.1300,1543251600,40.49,1543233600,47.30,1543251600,36.20,1543291200,43.92,1543251600
2,981a3613-77af-4620-a42a-0c0866077d1e,1.543367e+09,1,28,11,2018-11-28 01:00:22,America/New_York,Haymarket Square,North Station,Lyft,...,0.1064,1543338000,35.36,1543377600,47.55,1543320000,31.04,1543377600,44.12,1543320000
3,c2d88af2-d278-4bfd-a8d0-29ca77cc5512,1.543554e+09,4,30,11,2018-11-30 04:53:02,America/New_York,Haymarket Square,North Station,Lyft,...,0.0000,1543507200,34.67,1543550400,45.03,1543510800,30.30,1543550400,38.53,1543510800
4,e0126e1f-8ca9-4f2e-82b3-50505a09db9a,1.543463e+09,3,29,11,2018-11-29 03:49:20,America/New_York,Haymarket Square,North Station,Lyft,...,0.0001,1543420800,33.10,1543402800,42.18,1543420800,29.11,1543392000,35.75,1543420800


Data Cleaning

In [3]:
#Identify number of null values per column in the database
df.isna().sum()

id                                 0
timestamp                          0
hour                               0
day                                0
month                              0
datetime                           0
timezone                           0
source                             0
destination                        0
cab_type                           0
product_id                         0
name                               0
price                          55095
distance                           0
surge_multiplier                   0
latitude                           0
longitude                          0
temperature                        0
apparentTemperature                0
short_summary                      0
long_summary                       0
precipIntensity                    0
precipProbability                  0
humidity                           0
windSpeed                          0
windGust                           0
windGustTime                       0
v

In [4]:
#Drop rows that have a null value in the price column
df = df[df['price'].notna()]

In [5]:
#Remove duplicates in the ID column
df = df.drop_duplicates('id')

In [6]:
# Check for duplicate rows
duplicates = df.duplicated()
duplicates_num = duplicates.sum()

print(duplicates_num)

0


In [7]:
# Removing outliers in the price, ...

In [8]:
# Reduce data to only contain longitude between -180 and 180, and latitude between -90 and 90
df = df[(df['longitude'] > -180) & (df['longitude'] < 180)]
df = df[(df['latitude'] > -90) & (df['latitude'] < 90)]

In [9]:
# Separate the Datetime column into time, year, month, date and day of the week

df['datetime'] = pd.to_datetime(df['datetime'])

df['time'] = df['datetime'].dt.strftime('%H:%M')
df['year'] = df['datetime'].dt.year
df['month'] = df['datetime'].dt.strftime('%B')
df['day'] = df['datetime'].dt.day
df['day_of_week'] = df['datetime'].dt.dayofweek 

day_names = {0: 'Monday', 1: 'Tuesday', 2: 'Wednesday', 3: 'Thursday', 4: 'Friday', 5: 'Saturday', 6: 'Sunday'}
df['day_of_week'] = df['day_of_week'].map(day_names)

df.head()

,id,timestamp,hour,day,month,datetime,timezone,source,destination,cab_type,...,temperatureMinTime,temperatureMax,temperatureMaxTime,apparentTemperatureMin,apparentTemperatureMinTime,apparentTemperatureMax,apparentTemperatureMaxTime,time,year,day_of_week
0,424553bb-7174-41ea-aeb4-fe06d4f4b9d7,1.544953e+09,9,16,December,2018-12-16 09:30:07,America/New_York,Haymarket Square,North Station,Lyft,...,1545012000,43.68,1544968800,33.73,1545012000,38.07,1544958000,09:30,2018,Sunday
1,4bd23055-6827-41c6-b23b-3c491f24e74d,1.543284e+09,2,27,November,2018-11-27 02:00:23,America/New_York,Haymarket Square,North Station,Lyft,...,1543233600,47.30,1543251600,36.20,1543291200,43.92,1543251600,02:00,2018,Tuesday
2,981a3613-77af-4620-a42a-0c0866077d1e,1.543367e+09,1,28,November,2018-11-28 01:00:22,America/New_York,Haymarket Square,North Station,Lyft,...,1543377600,47.55,1543320000,31.04,1543377600,44.12,1543320000,01:00,2018,Wednesday
3,c2d88af2-d278-4bfd-a8d0-29ca77cc5512,1.543554e+09,4,30,November,2018-11-30 04:53:02,America/New_York,Haymarket Square,North Station,Lyft,...,1543550400,45.03,1543510800,30.30,1543550400,38.53,1543510800,04:53,2018,Friday
4,e0126e1f-8ca9-4f2e-82b3-50505a09db9a,1.543463e+09,3,29,November,2018-11-29 03:49:20,America/New_York,Haymarket Square,North Station,Lyft,...,1543402800,42.18,1543420800,29.11,1543392000,35.75,1543420800,03:49,2018,Thursday
